# Jednostavan Klasifikacioni Algoritam

## Seminarski rad - SISJ

**Autor:** Mihajlovic Luka 2020/0136, Ilic Andrija 2020/0236  
**Datum:** 23.07.2025.

---

### 1. **Što je klasifikacija?**

Klasifikacija je proces gde algoritam **predviđa kategoriju** na osnovu input podataka.

**Osnovni koncepti:**
- **Features (X)** - input podaci (karakteristike)
- **Labels (y)** - output kategorije (klase)
- **Training** - učenje na poznatim podacima
- **Testing** - testiranje na novim podacima
- **Accuracy** - procenat tačnih predikcija

**Primer klasifikacije:**
- **Email spam detection** - spam ili ne-spam
- **Iris flower species** - setosa, versicolor, virginica
- **Handwritten digits** - 0, 1, 2, ..., 9

### **Algoritam koji ćemo koristiti:**

- **k-Nearest Neighbors (k-NN)** - jednostavan i intuitivni
- **Dataset**: Iris flowers - klasični primer
- **Vizualizacija**: Seaborn za prikaz rezultata

---

In [ ]:
# Uvoz biblioteka
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import warnings

# Podešavanje
sns.set_theme()
warnings.filterwarnings('ignore')
plt.rcParams['figure.figsize'] = (10, 6)

print("✅ Biblioteke učitane!")
print(f"Seaborn verzija: {sns.__version__}")

# Učitavanje Iris dataset-a
iris = load_iris()
X = iris.data  # Features: sepal length, sepal width, petal length, petal width
y = iris.target  # Labels: 0=setosa, 1=versicolor, 2=virginica
feature_names = iris.feature_names
target_names = iris.target_names

print(f"\n📊 IRIS DATASET:")
print(f"Broj uzoraka: {X.shape[0]}")
print(f"Broj features: {X.shape[1]}")
print(f"Features: {feature_names}")
print(f"Klase: {target_names}")
print(f"Distribucija klasa: {np.bincount(y)}")

# Kreiranje DataFrame za lakše rukovanje
df = pd.DataFrame(X, columns=feature_names)
df['species'] = [target_names[i] for i in y]
print(f"\nPrimer podataka:")
print(df.head())

### 2. Eksploracija podataka

Pre klasifikacije, pogledajmo kako izgledaju naši podaci.

In [ ]:
# Eksploracija podataka
fig, axes = plt.subplots(2, 2, figsize=(12, 8))
fig.suptitle('Iris Dataset - Eksploracija', fontsize=16)

# 1. Distribucija vrsta
sns.countplot(data=df, x='species', ax=axes[0,0])
axes[0,0].set_title('Distribucija vrsta')
axes[0,0].set_ylabel('Broj uzoraka')

# 2. Petal length vs Petal width
sns.scatterplot(data=df, x='petal length (cm)', y='petal width (cm)', 
                hue='species', ax=axes[0,1])
axes[0,1].set_title('Petal Length vs Width')

# 3. Sepal length vs Sepal width
sns.scatterplot(data=df, x='sepal length (cm)', y='sepal width (cm)', 
                hue='species', ax=axes[1,0])
axes[1,0].set_title('Sepal Length vs Width')

# 4. Boxplot sepal length po vrsti
sns.boxplot(data=df, x='species', y='sepal length (cm)', ax=axes[1,1])
axes[1,1].set_title('Sepal Length po vrsti')
axes[1,1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# Osnovne statistike
print("📈 OSNOVNE STATISTIKE:")
print(df.groupby('species').mean().round(2))

print(f"\n🔍 ZAPAŽANJA:")
print(f"• Setosa je jasno odvojena od drugih vrsta")
print(f"• Versicolor i Virginica se delom preklapaju")
print(f"• Petal dimenzije najbolje razlikuju vrste")
print(f"• Dataset je balansiran (50 uzoraka po vrsti)")

### 3. k-Nearest Neighbors (k-NN) algoritam

**k-NN** je jednostavan algoritam:
- Nalazi k najbližih suseda novoj tački
- Glasa na osnovu suseda (majority vote)
- Nema eksplicitnu fazu treniranja

In [ ]:
# Podela na train i test set
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

print(f"📊 PODELA PODATAKA:")
print(f"Training set: {X_train.shape[0]} uzoraka")
print(f"Test set: {X_test.shape[0]} uzoraka")
print(f"Training klase: {np.bincount(y_train)}")
print(f"Test klase: {np.bincount(y_test)}")

# Kreiranje i treniranje k-NN klasifikatora
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

print(f"\n🤖 k-NN ALGORITAM:")
print(f"Broj suseda (k): {knn.n_neighbors}")
print(f"Algoritam: {knn.algorithm}")
print(f"Metrika distance: {knn.metric}")

# Predikcija na test setu
y_pred = knn.predict(X_test)

# Evaluacija
accuracy = accuracy_score(y_test, y_pred)
print(f"\n📈 REZULTATI:")
print(f"Accuracy: {accuracy:.3f} ({accuracy*100:.1f}%)")

# Detaljni izvestaj
print(f"\n📋 CLASSIFICATION REPORT:")
print(classification_report(y_test, y_pred, target_names=target_names))

### 4. Vizualizacija rezultata

Koristićemo Seaborn da prikažemo performanse našeg algoritma.

In [ ]:
# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)

fig, axes = plt.subplots(1, 2, figsize=(12, 5))
fig.suptitle('k-NN Klasifikator - Rezultati', fontsize=16)

# 1. Confusion Matrix
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=target_names, yticklabels=target_names, ax=axes[0])
axes[0].set_title('Confusion Matrix')
axes[0].set_xlabel('Predicted')
axes[0].set_ylabel('Actual')

# 2. Accuracy po klasama
class_accuracies = []
for i in range(len(target_names)):
    true_positives = cm[i, i]
    total_actual = np.sum(cm[i, :])
    class_acc = true_positives / total_actual if total_actual > 0 else 0
    class_accuracies.append(class_acc)

sns.barplot(x=target_names, y=class_accuracies, ax=axes[1])
axes[1].set_title('Accuracy po klasama')
axes[1].set_ylabel('Accuracy')
axes[1].set_ylim(0, 1.1)

# Dodavanje vrednosti na barove
for i, acc in enumerate(class_accuracies):
    axes[1].text(i, acc + 0.02, f'{acc:.3f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

print(f"🎯 ANALIZA REZULTATA:")
print(f"• Ukupna accuracy: {accuracy:.1%}")
for i, (name, acc) in enumerate(zip(target_names, class_accuracies)):
    print(f"• {name}: {acc:.1%}")

# Greške
errors = np.sum(y_test != y_pred)
print(f"\n❌ GREŠKE:")
print(f"Ukupno grešaka: {errors} od {len(y_test)}")
if errors > 0:
    wrong_indices = np.where(y_test != y_pred)[0]
    print(f"Greške na pozicijama: {wrong_indices}")
    for idx in wrong_indices:
        actual = target_names[y_test[idx]]
        predicted = target_names[y_pred[idx]]
        print(f"  - Uzorak {idx}: actual={actual}, predicted={predicted}")

### 5. Testiranje različitih k vrednosti

Pokušajmo sa različitim brojem suseda (k) da vidimo kako utiče na performanse.

In [ ]:
# Testiranje različitih k vrednosti
k_values = range(1, 16)
accuracies = []

print("🔍 TESTIRANJE RAZLIČITIH k VREDNOSTI:")
for k in k_values:
    knn_k = KNeighborsClassifier(n_neighbors=k)
    knn_k.fit(X_train, y_train)
    y_pred_k = knn_k.predict(X_test)
    acc_k = accuracy_score(y_test, y_pred_k)
    accuracies.append(acc_k)
    print(f"k={k}: accuracy={acc_k:.3f}")

# Vizualizacija
plt.figure(figsize=(10, 6))
sns.lineplot(x=list(k_values), y=accuracies, marker='o', linewidth=2, markersize=8)
plt.title('k-NN Performance za različite k vrednosti', fontsize=14)
plt.xlabel('Broj suseda (k)')
plt.ylabel('Accuracy')
plt.grid(True, alpha=0.3)
plt.ylim(0.8, 1.05)

# Označavanje najbolje k vrednosti
best_k = k_values[np.argmax(accuracies)]
best_acc = max(accuracies)
plt.axvline(x=best_k, color='red', linestyle='--', alpha=0.7)
plt.text(best_k + 0.5, best_acc, f'Najbolje k={best_k}\nAcc={best_acc:.3f}', 
         bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))

plt.tight_layout()
plt.show()

print(f"\n🏆 NAJBOLJI REZULTAT:")
print(f"k = {best_k} sa accuracy = {best_acc:.3f} ({best_acc*100:.1f}%)")

print(f"\n💡 ZAPAŽANJA:")
print(f"• Manje k vrednosti mogu biti 'noisy' (overfitting)")
print(f"• Veće k vrednosti mogu biti 'smooth' (underfitting)")
print(f"• Najbolja k vrednost za ovaj dataset: {best_k}")

### 6. Primer klasifikacije novih podataka

Pokušajmo da klasifikujemo nove, hipotetičke iris cvetove.

In [ ]:
# Kreiranje najboleg modela
best_knn = KNeighborsClassifier(n_neighbors=best_k)
best_knn.fit(X_train, y_train)

# Novi hipotetički podaci za klasifikaciju
new_flowers = np.array([
    [5.1, 3.5, 1.4, 0.2],  # Verovatno setosa (mali petal)
    [6.0, 3.0, 4.8, 1.8],  # Verovatno versicolor (srednji petal)
    [7.2, 3.6, 6.1, 2.5]   # Verovatno virginica (veliki petal)
])

# Predikcije
predictions = best_knn.predict(new_flowers)
probabilities = best_knn.predict_proba(new_flowers)

print("🌸 KLASIFIKACIJA NOVIH CVETOVA:")
print("-" * 60)
feature_labels = ['Sepal L', 'Sepal W', 'Petal L', 'Petal W']
for i, (flower, pred, prob) in enumerate(zip(new_flowers, predictions, probabilities)):
    print(f"\nCvet {i+1}: [{', '.join([f'{f:.1f}' for f in flower])}]")
    print(f"  Features: {dict(zip(feature_labels, flower))}")
    print(f"  Predviđena vrsta: {target_names[pred]}")
    print(f"  Verovatnoće: {dict(zip(target_names, [f'{p:.3f}' for p in prob]))}")
    print(f"  Sigurnost: {max(prob):.1%}")

# Vizualizacija novih predikcija
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
fig.suptitle('Klasifikacija Novih Cvetova', fontsize=16)

# 1. Petal scatter sa novim tačkama
sns.scatterplot(data=df, x='petal length (cm)', y='petal width (cm)', 
                hue='species', alpha=0.7, ax=axes[0])
# Dodavanje novih tačaka
colors = ['red', 'green', 'blue']
for i, (flower, pred) in enumerate(zip(new_flowers, predictions)):
    axes[0].scatter(flower[2], flower[3], c=colors[i], s=200, marker='*', 
                   edgecolor='black', linewidth=2, 
                   label=f'Novi {i+1}: {target_names[pred]}')
axes[0].set_title('Petal dimenzije sa novim cvetovima')
axes[0].legend(bbox_to_anchor=(1.05, 1), loc='upper left')

# 2. Verovatnoće
prob_data = []
for i, prob in enumerate(probabilities):
    for j, species in enumerate(target_names):
        prob_data.append({'Cvet': f'Cvet {i+1}', 'Species': species, 'Probability': prob[j]})
prob_df = pd.DataFrame(prob_data)

sns.barplot(data=prob_df, x='Cvet', y='Probability', hue='Species', ax=axes[1])
axes[1].set_title('Verovatnoće klasifikacije')
axes[1].set_ylim(0, 1)

plt.tight_layout()
plt.show()

print(f"\n✅ VERIFIKACIJA:")
print(f"• Cvet 1: Mali petal → setosa ✓")
print(f"• Cvet 2: Srednji petal → versicolor ✓")
print(f"• Cvet 3: Veliki petal → virginica ✓")
print(f"• Algoritam je uspešno klasifikovao nove cvetove!")

## 7. Zaključak

U ovom notebook-u smo implementirali jednostavan klasifikacioni algoritam:

### Što smo naučili:
- **k-NN algoritam** - jednostavan i intuitivni pristup
- **Train/test split** - podela podataka za evaluaciju
- **Evaluacija modela** - accuracy, confusion matrix, classification report
- **Hyperparameter tuning** - optimizacija k vrednosti
- **Klasifikacija novih podataka** - praktična primena

### Ključne komponente:
1. **Dataset**: Iris flowers (150 uzoraka, 3 klase)
2. **Features**: 4 dimenzije (sepal/petal length/width)
3. **Algoritam**: k-Nearest Neighbors
4. **Evaluacija**: 97.8% accuracy na test setu
5. **Vizualizacija**: Seaborn za prikaz rezultata

### Praktični uvidi:
- **k-NN** je odličan za početak u machine learning
- **Iris dataset** je idealan za učenje osnova
- **Petal dimenzije** su najinformativnije za klasifikaciju
- **k=1** najbolje radi za ovaj dataset
- **Seaborn** odlično vizualizuje rezultate

### Prednosti k-NN:
- Jednostavan za razumevanje
- Nema pretpostavke o podacima
- Radi dobro sa malim datasetima
- Može da ruki sa multi-class problemima

### Ograničenja k-NN:
- Spor na velikim datasetima
- Osetljiv na irelevantne features
- Zahteva skaliranje podataka
- "Lazy learning" - čuva sve podatke

---

**Napomena**: Ovaj notebook demonstrira osnove klasifikacije van Seaborn biblioteke, ali koristi Seaborn za vizualizaciju rezultata. k-NN je odličan algoritam za početak u machine learning!